# 病床姿勢監測 - 資料準備與對齊（修正版）

**專案**: 熱成像超解析度 + 姿勢辨識  
**作者**: rochi190  
**日期**: 2025-11-03  

## 功能
1. 讀取 output.txt 取得相機開始時間
2. 載入熱成像 log 檔案（多核加速）
3. 分析 RGB 影片資訊
4. **Adaptive 配對策略**（處理不固定 FPS 和資料遺漏）：
   - 分析每秒幀數分佈
   - 檢測 burst periods（>8 張/秒）和 dropout（<8 張/秒）
   - 將 burst 多餘的幀重新分配給 dropout 區段
   - 只要時間匹配就配對（不強制 8fps）
5. 翻轉檢測與修復
6. 匯出 RGB 圖片供 LabelMe 標註（多核加速）
7. 儲存對齊後的 Thermal 資料

## 輸出
- `output/labelme_project/` - LabelMe 標註專案
- `output/aligned_dataset/` - 對齊後的訓練資料
- `output/adaptive_pairing/` - 配對分析結果

---
## Step 0: 環境設定與套件匯入

In [34]:
import os
import sys
import json
import re
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import datetime, timedelta
from typing import List, Dict, Tuple, Optional, Any
from tqdm import tqdm
from dataclasses import dataclass, field
from concurrent.futures import ProcessPoolExecutor
from collections import defaultdict
import multiprocessing as mp
import warnings
warnings.filterwarnings('ignore')

# Matplotlib 設定
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.dpi'] = 100

# Seaborn 樣式
sns.set_style('whitegrid')

print("="*70)
print("病床姿勢監測系統 - 資料準備模組（Adaptive Pairing）")
print("="*70)
print(f"執行時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"使用者: rochi190")
print(f"Python 版本: {sys.version}")
print(f"NumPy 版本: {np.__version__}")
print(f"OpenCV 版本: {cv2.__version__}")
print(f"CPU 核心數: {mp.cpu_count()}")
print("="*70)

病床姿勢監測系統 - 資料準備模組（Adaptive Pairing）
執行時間: 2025-11-04 20:41:37
使用者: rochi190
Python 版本: 3.11.13 (main, Jun  5 2025, 13:12:00) [GCC 11.2.0]
NumPy 版本: 1.26.4
OpenCV 版本: 4.10.0
CPU 核心數: 4


---
## Step 1: 路徑與參數配置

In [35]:
# ========================================
# 多核處理配置
# ========================================
NUM_WORKERS = max(1, mp.cpu_count() - 1)
CHUNK_SIZE = 100

print(f"🚀 多核處理配置:")
print(f"  使用 Workers: {NUM_WORKERS}")
print(f"  批次大小: {CHUNK_SIZE}")

# ========================================
# 路徑配置
# ========================================
BASE_DIR = Path(os.getcwd())
DATA_DIR = BASE_DIR / 'data'
THERMAL_DIR = DATA_DIR / 'thermal'
CAMERA_DIR = DATA_DIR / 'camera'
OUTPUT_TXT = CAMERA_DIR / 'output.txt'
AVI_FILE = CAMERA_DIR / 'output.avi'

# 輸出目錄
OUTPUT_DIR = BASE_DIR / 'output'
ADAPTIVE_DIR = OUTPUT_DIR / 'adaptive_pairing'
LABELME_DIR = OUTPUT_DIR / 'labelme_project'
ALIGNED_DIR = OUTPUT_DIR / 'aligned_dataset'
DIAGNOSTIC_DIR = OUTPUT_DIR / 'diagnostics'

for directory in [ADAPTIVE_DIR, LABELME_DIR, ALIGNED_DIR, DIAGNOSTIC_DIR]:
    directory.mkdir(parents=True, exist_ok=True)

# ========================================
# 熱成像參數
# ========================================
THERMAL_RESOLUTION = (32, 24)  # (寬度, 高度)
THERMAL_WIDTH, THERMAL_HEIGHT = THERMAL_RESOLUTION

# ========================================
# Adaptive 配對參數
# ========================================
TARGET_FPS = 8  # 目標 FPS（僅供參考，不強制）
NORMAL_FRAMES_PER_SEC = 8  # 正常每秒幀數

# 翻轉設定（執行 Step 6 後調整）
FLIP_THERMAL = True
FLIP_RGB = False

# ========================================
# 顯示配置
# ========================================
print(f"\n📁 路徑檢查:")
print(f"  Thermal 目錄: {'✓' if THERMAL_DIR.exists() else '✗'} {THERMAL_DIR}")
print(f"  Camera 目錄: {'✓' if CAMERA_DIR.exists() else '✗'} {CAMERA_DIR}")
print(f"  output.txt: {'✓' if OUTPUT_TXT.exists() else '✗'} {OUTPUT_TXT}")
print(f"  AVI 檔案: {'✓' if AVI_FILE.exists() else '✗'} {AVI_FILE}")

print(f"\n⚙️ 配對參數:")
print(f"  目標 FPS: {TARGET_FPS}（不強制）")
print(f"  正常幀數/秒: {NORMAL_FRAMES_PER_SEC}")

print(f"\n🔄 翻轉設定:")
print(f"  翻轉 Thermal: {FLIP_THERMAL}")
print(f"  翻轉 RGB: {FLIP_RGB}")

🚀 多核處理配置:
  使用 Workers: 3
  批次大小: 100

📁 路徑檢查:
  Thermal 目錄: ✓ /home/gary/claude4.5/data/thermal
  Camera 目錄: ✓ /home/gary/claude4.5/data/camera
  output.txt: ✓ /home/gary/claude4.5/data/camera/output.txt
  AVI 檔案: ✓ /home/gary/claude4.5/data/camera/output.avi

⚙️ 配對參數:
  目標 FPS: 8（不強制）
  正常幀數/秒: 8

🔄 翻轉設定:
  翻轉 Thermal: True
  翻轉 RGB: False


---
## Step 2: 解析 output.txt（相機開始時間）

In [36]:
def parse_output_txt(txt_path: Path) -> Optional[datetime]:
    """
    解析 output.txt 取得相機開始時間
    """
    if not txt_path.exists():
        print(f"❌ {txt_path} 不存在")
        return None
    
    try:
        with open(txt_path, 'r', encoding='utf-8') as f:
            content = f.read()
    except:
        with open(txt_path, 'r', encoding='big5') as f:
            content = f.read()
    
    print(f"\n{'='*70}")
    print(f"解析 output.txt")
    print(f"{'='*70}")
    print(f"檔案大小: {len(content)} bytes")
    print(f"\n前 500 字元:\n{content[:500]}")
    print(f"{'='*70}")
    
    # 時間格式模式
    patterns = [
        (r'(\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2})', '%Y-%m-%d %H:%M:%S'),
        (r'(\d{4}/\d{2}/\d{2}\s+\d{2}:\d{2}:\d{2})', '%Y/%m/%d %H:%M:%S'),
    ]
    
    for pattern, fmt in patterns:
        match = re.search(pattern, content)
        if match:
            try:
                camera_start = datetime.strptime(match.group(1), fmt)
                print(f"\n✅ 找到相機開始時間: {camera_start}")
                return camera_start
            except:
                continue
    
    print(f"\n❌ 無法解析時間")
    return None

# 執行解析
camera_start_time = parse_output_txt(OUTPUT_TXT)

if camera_start_time is None:
    raise RuntimeError("❌ 無法解析 output.txt，請檢查檔案格式")

print(f"\n📌 相機開始時間: {camera_start_time}")


解析 output.txt
檔案大小: 899 bytes

前 500 字元:
viedo_start_2020-09-24 13:09:29 
lay_mid_2020-09-24 13:11:08 
end_time_2020-09-24 13:11:43 
lay_right_2020-09-24 13:12:02 
end_time_2020-09-24 13:12:39 
lay_left_2020-09-24 13:12:58 
end_time_2020-09-24 13:13:36 
bed_2020-09-24 13:13:53 
end_time_2020-09-24 13:14:32 
sit_right_2020-09-24 13:14:40 
end_time_2020-09-24 13:15:22 
sit_left_2020-09-24 13:15:27 
end_time_2020-09-24 13:16:04 
Falldown_2020-09-24 13:16:38 
end_time_2020-09-24 13:16:43 
Falldown_2020-09-24 13:16:57 
end_time_2020-09-24 1

✅ 找到相機開始時間: 2020-09-24 13:09:29

📌 相機開始時間: 2020-09-24 13:09:29


---
## Step 3: 載入熱成像資料（多核加速）

In [37]:
def parse_thermal_file(file_path: Path) -> List[Dict]:
    """
    解析單一熱成像 log 檔案（用於多核處理）
    """
    data = []
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if not line:
                    continue
                
                try:
                    data_dict = json.loads(line)
                    time_parts = data_dict.get('time', [])
                    
                    if len(time_parts) < 6:
                        continue
                    
                    year, month, day, hour, minute, second = time_parts[:6]
                    millisecond = time_parts[7] if len(time_parts) > 7 else 0
                    
                    timestamp = datetime(year, month, day, hour, minute, second, millisecond * 1000)
                    
                    message = data_dict.get('message', [])
                    if len(message) != THERMAL_WIDTH * THERMAL_HEIGHT:
                        continue
                    
                    thermal_image = np.array(message, dtype=np.uint8).reshape(
                        THERMAL_HEIGHT, THERMAL_WIDTH
                    )
                    
                    data.append({
                        'timestamp': timestamp,
                        'image': thermal_image
                    })
                    
                except:
                    continue
    
    except Exception as e:
        print(f"❌ 載入失敗 {file_path.name}: {e}")
        return []
    
    return data

# 載入所有 thermal log
log_files = sorted(THERMAL_DIR.glob('log*.txt'))

if not log_files:
    raise FileNotFoundError(f"❌ 未找到 log 檔案於 {THERMAL_DIR}")

print(f"\n{'='*70}")
print(f"載入熱成像資料（多核: {NUM_WORKERS} workers）")
print(f"{'='*70}")
print(f"找到 {len(log_files)} 個 log 檔案")

therm_data = []
with ProcessPoolExecutor(max_workers=NUM_WORKERS) as executor:
    results = list(tqdm(
        executor.map(parse_thermal_file, log_files),
        total=len(log_files),
        desc="載入 Thermal"
    ))

for file_data in results:
    therm_data.extend(file_data)

# 排序
therm_data.sort(key=lambda x: x['timestamp'])

print(f"\n✅ 載入完成")
print(f"  總幀數: {len(therm_data):,}")
if therm_data:
    thermal_start = therm_data[0]['timestamp']
    thermal_end = therm_data[-1]['timestamp']
    duration = (thermal_end - thermal_start).total_seconds()
    print(f"  時間範圍: {thermal_start} ~ {thermal_end}")
    print(f"  時長: {duration:.2f} 秒")
    print(f"  平均 FPS: {len(therm_data) / duration:.2f}")
print("="*70)


載入熱成像資料（多核: 3 workers）
找到 5 個 log 檔案


載入 Thermal: 100%|██████████| 5/5 [00:00<00:00, 22.08it/s]


✅ 載入完成
  總幀數: 5,318
  時間範圍: 2020-09-24 13:09:30.268000 ~ 2020-09-24 13:20:52.268000
  時長: 682.00 秒
  平均 FPS: 7.80


---
## Step 4: 分析 RGB 影片

In [38]:
# 分析 AVI 影片
cap = cv2.VideoCapture(str(AVI_FILE))

if not cap.isOpened():
    raise RuntimeError(f"❌ 無法開啟影片 {AVI_FILE}")

fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
duration = frame_count / fps if fps > 0 else 0

cap.release()

print(f"\n{'='*70}")
print(f"RGB 影片資訊")
print(f"{'='*70}")
print(f"  FPS: {fps}")
print(f"  總幀數: {frame_count:,}")
print(f"  解析度: {width} × {height}")
print(f"  時長: {duration:.2f} 秒")
print(f"  開始時間: {camera_start_time}")
print(f"  結束時間: {camera_start_time + timedelta(seconds=duration)}")
print("="*70)


RGB 影片資訊
  FPS: 24.67
  總幀數: 16,284
  解析度: 640 × 360
  時長: 660.07 秒
  開始時間: 2020-09-24 13:09:29
  結束時間: 2020-09-24 13:20:29.072963


---
## Step 5: Adaptive 配對策略（核心演算法）

In [39]:
def analyze_thermal_distribution(thermal_data: List[Dict], camera_start_time: datetime) -> Dict:
    """
    分析熱成像資料的時間分佈
    將幀按秒分組
    """
    sections = {}
    
    for frame in thermal_data:
        offset = (frame['timestamp'] - camera_start_time).total_seconds()
        
        if offset < 0:
            continue
        
        second = int(offset)
        if second not in sections:
            sections[second] = []
        sections[second].append(frame)
    
    print(f"\n{'='*70}")
    print(f"熱成像時間分佈分析")
    print(f"{'='*70}")
    
    section_counts = {sec: len(frames) for sec, frames in sections.items()}
    counts = list(section_counts.values())
    
    print(f"\n總秒數: {len(sections)}")
    print(f"最小幀數/秒: {min(counts)}")
    print(f"最大幀數/秒: {max(counts)}")
    print(f"平均幀數/秒: {np.mean(counts):.1f}")
    print(f"標準差: {np.std(counts):.1f}")
    
    # 分類
    normal_seconds = [sec for sec, count in section_counts.items() if count == NORMAL_FRAMES_PER_SEC]
    dropout_seconds = [sec for sec, count in section_counts.items() if count < NORMAL_FRAMES_PER_SEC]
    burst_seconds = {sec: count for sec, count in section_counts.items() if count > NORMAL_FRAMES_PER_SEC}
    
    print(f"\n正常 ({NORMAL_FRAMES_PER_SEC} 張/秒): {len(normal_seconds)} 秒")
    print(f"Dropout (<{NORMAL_FRAMES_PER_SEC} 張/秒): {len(dropout_seconds)} 秒")
    print(f"Burst (>{NORMAL_FRAMES_PER_SEC} 張/秒): {len(burst_seconds)} 秒")
    
    if burst_seconds:
        print(f"\nBurst 範例（前 5 秒）:")
        for sec, count in sorted(burst_seconds.items(), key=lambda x: x[1], reverse=True)[:5]:
            print(f"  第 {sec} 秒: {count} 張 (多 {count - NORMAL_FRAMES_PER_SEC} 張)")
    
    print("="*70)
    
    return sections

# 執行分析
sections = analyze_thermal_distribution(therm_data, camera_start_time)


熱成像時間分佈分析

總秒數: 677
最小幀數/秒: 5
最大幀數/秒: 57
平均幀數/秒: 7.9
標準差: 2.0

正常 (8 張/秒): 462 秒
Dropout (<8 張/秒): 178 秒
Burst (>8 張/秒): 37 秒

Burst 範例（前 5 秒）:
  第 670 秒: 57 張 (多 49 張)
  第 8 秒: 9 張 (多 1 張)
  第 12 秒: 9 張 (多 1 張)
  第 87 秒: 9 張 (多 1 張)
  第 92 秒: 9 張 (多 1 張)


In [40]:
def adaptive_thermal_rgb_pairing(
    thermal_data: List[Dict],
    camera_start_time: datetime,
    fps: float,
    frame_count: int,
    sections: Dict
) -> Tuple[List[Dict], List[Dict]]:
    """
    Adaptive 配對策略（修正版）
    
    步驟：
    1. 找出 burst seconds（>8 張/秒）和 dropout seconds（<8 張/秒）
    2. 收集 burst 中多餘的幀（保留「後 8 張」，前面的是延遲資料）
    3. 重新分配給 dropout seconds
    4. 配對 Thermal 和 RGB
    """
    print(f"\n{'='*70}")
    print(f"Adaptive Thermal-RGB 配對（修正版）")
    print(f"{'='*70}")
    
    # 1. 分類 burst 和 dropout
    burst_seconds = {}
    dropout_seconds = []
    
    for sec in sorted(sections.keys()):
        count = len(sections[sec])
        if count > NORMAL_FRAMES_PER_SEC:
            burst_seconds[sec] = count - NORMAL_FRAMES_PER_SEC
        elif count < NORMAL_FRAMES_PER_SEC:
            dropout_seconds.append(sec)
    
    print(f"\n1. 重新分配策略:")
    print(f"   Burst seconds: {len(burst_seconds)}")
    print(f"   Dropout seconds: {len(dropout_seconds)}")
    
    # 2. 收集 burst 中多餘的幀
    # 🔧 修正：保留「後 8 張」（最新的），前面的是延遲資料
    excess_frames = []
    for sec in sorted(burst_seconds.keys()):
        frames = sections[sec]
        
        # 保留後 8 張（最新的資料）
        # 前面的是延遲資料，加入 excess 重新分配
        delayed_frames = frames[:-NORMAL_FRAMES_PER_SEC] if len(frames) > NORMAL_FRAMES_PER_SEC else []
        
        for frame in delayed_frames:
            excess_frames.append((frame, sec))
        
        # 🔧 更新 sections：只保留後 8 張
        sections[sec] = frames[-NORMAL_FRAMES_PER_SEC:]
    
    print(f"\n2. 多餘幀收集:")
    print(f"   總延遲幀數: {len(excess_frames)}")
    
    # 3. 重新分配給 dropout
    # 🔧 優化：按時間順序重新分配（延遲的幀應該回到它原本的時間）
    excess_idx = 0
    redistribution_log = []
    
    for dropout_sec in sorted(dropout_seconds):
        missing_count = NORMAL_FRAMES_PER_SEC - len(sections[dropout_sec])
        
        if excess_idx < len(excess_frames):
            frames_to_add = min(missing_count, len(excess_frames) - excess_idx)
            
            for i in range(frames_to_add):
                frame, original_sec = excess_frames[excess_idx]
                sections[dropout_sec].append(frame)
                
                redistribution_log.append({
                    'from_sec': original_sec,
                    'to_sec': dropout_sec,
                    'frame': frame
                })
                
                excess_idx += 1
    
    print(f"   重新分配幀數: {len(redistribution_log)}")
    
    # 🔧 改進：按時間戳排序重新分配後的 sections
    for sec in sections.keys():
        sections[sec].sort(key=lambda x: x['timestamp'])
    
    # 4. 配對 Thermal 和 RGB
    pairs = []
    rgb_end_time = camera_start_time + timedelta(seconds=frame_count / fps)
    
    for sec in sorted(sections.keys()):
        for thermal_frame in sections[sec]:
            thermal_ts = thermal_frame['timestamp']
            
            # 檢查是否在 RGB 範圍內
            if thermal_ts < camera_start_time or thermal_ts > rgb_end_time:
                continue
            
            # 計算 RGB 幀索引
            offset = (thermal_ts - camera_start_time).total_seconds()
            rgb_frame_idx = int(round(offset * fps))
            rgb_frame_idx = max(0, min(rgb_frame_idx, frame_count - 1))
            
            # 計算誤差
            actual_time = rgb_frame_idx / fps
            rgb_error = abs(actual_time - offset)
            
            pairs.append({
                'pair_id': f'pair_{len(pairs):05d}',
                'thermal': thermal_frame,
                'rgb_frame_idx': rgb_frame_idx,
                'timestamp': thermal_ts,
                'section': sec,
                'rgb_error_ms': rgb_error * 1000
            })
    
    print(f"\n3. 最終配對結果:")
    print(f"   總配對數: {len(pairs):,}")
    print(f"   成功率: {len(pairs) / len(thermal_data) * 100:.1f}%")
    
    if pairs:
        rgb_errors = [p['rgb_error_ms'] for p in pairs]
        print(f"\n   RGB 誤差統計:")
        print(f"   平均: {np.mean(rgb_errors):.2f}ms")
        print(f"   中位數: {np.median(rgb_errors):.2f}ms")
        print(f"   最大: {np.max(rgb_errors):.2f}ms")
        print(f"   標準差: {np.std(rgb_errors):.2f}ms")
    
    print("="*70)
    
    return pairs, redistribution_log

# 執行配對
pairs_adaptive, redistribution_log = adaptive_thermal_rgb_pairing(
    therm_data, camera_start_time, fps, frame_count, sections
)

print(f"\n📌 配對結果: {len(pairs_adaptive):,} 對")

# ========================================
# 按 Thermal 時間戳排序（確保時間連續）
# ========================================

print(f"\n{'='*70}")
print(f"按 Thermal 時間戳排序配對")
print(f"{'='*70}")

print(f"排序前範例（前 10 對）:")
for i, pair in enumerate(pairs_adaptive[:10]):
    thermal_time = pair['timestamp'].strftime('%H:%M:%S.%f')[:-3]
    print(f"  {pair['pair_id']}: Thermal {thermal_time}, RGB 幀 {pair['rgb_frame_idx']}")

# 按 Thermal 時間戳排序
pairs_sorted_by_time = sorted(pairs_adaptive, key=lambda x: x['timestamp'])

# 重新編號
for i, pair in enumerate(pairs_sorted_by_time):
    pair['pair_id'] = f'pair_{i:05d}'

pairs_adaptive = pairs_sorted_by_time

print(f"\n排序後範例（前 10 對）:")
for i, pair in enumerate(pairs_adaptive[:10]):
    thermal_time = pair['timestamp'].strftime('%H:%M:%S.%f')[:-3]
    print(f"  {pair['pair_id']}: Thermal {thermal_time}, RGB 幀 {pair['rgb_frame_idx']}")

# 檢查時間連續性
time_diffs = []
for i in range(len(pairs_adaptive) - 1):
    dt = (pairs_adaptive[i+1]['timestamp'] - pairs_adaptive[i]['timestamp']).total_seconds() * 1000
    time_diffs.append(dt)

print(f"\n✅ 時間排序完成")
print(f"  時間範圍: {pairs_adaptive[0]['timestamp'].strftime('%H:%M:%S')} ~ {pairs_adaptive[-1]['timestamp'].strftime('%H:%M:%S')}")
print(f"  平均時間間隔: {np.mean(time_diffs):.1f}ms")
print(f"  預期間隔 (8fps): {1000/8:.1f}ms")

print("="*70)

print(f"\n📌 配對結果（已按時間排序）: {len(pairs_adaptive):,} 對")


Adaptive Thermal-RGB 配對（修正版）

1. 重新分配策略:
   Burst seconds: 37
   Dropout seconds: 178

2. 多餘幀收集:
   總延遲幀數: 85
   重新分配幀數: 85

3. 最終配對結果:
   總配對數: 5,132
   成功率: 96.5%

   RGB 誤差統計:
   平均: 10.10ms
   中位數: 10.20ms
   最大: 20.20ms
   標準差: 5.86ms

📌 配對結果: 5,132 對

按 Thermal 時間戳排序配對
排序前範例（前 10 對）:
  pair_00000: Thermal 13:09:30.268, RGB 幀 31
  pair_00001: Thermal 13:09:30.268, RGB 幀 31
  pair_00002: Thermal 13:09:30.268, RGB 幀 31
  pair_00003: Thermal 13:09:30.268, RGB 幀 31
  pair_00004: Thermal 13:09:30.268, RGB 幀 31
  pair_00005: Thermal 13:09:37.268, RGB 幀 204
  pair_00006: Thermal 13:09:41.268, RGB 幀 303
  pair_00007: Thermal 13:10:56.268, RGB 幀 2153
  pair_00008: Thermal 13:09:31.268, RGB 幀 56
  pair_00009: Thermal 13:09:31.268, RGB 幀 56

排序後範例（前 10 對）:
  pair_00000: Thermal 13:09:30.268, RGB 幀 31
  pair_00001: Thermal 13:09:30.268, RGB 幀 31
  pair_00002: Thermal 13:09:30.268, RGB 幀 31
  pair_00003: Thermal 13:09:30.268, RGB 幀 31
  pair_00004: Thermal 13:09:30.268, RGB 幀 31
  pair_00005:

---
## Step 6: 配對品質驗證

In [41]:
# 驗證配對品質
print(f"\n{'='*70}")
print(f"配對品質驗證")
print(f"{'='*70}")

# 每秒配對數分佈
section_pairs = {}
for pair in pairs_adaptive:
    sec = pair['section']
    if sec not in section_pairs:
        section_pairs[sec] = 0
    section_pairs[sec] += 1

pair_counts = list(section_pairs.values())
print(f"\n每秒配對數分佈:")
print(f"  平均: {np.mean(pair_counts):.1f}")
print(f"  最小: {min(pair_counts)}")
print(f"  最大: {max(pair_counts)}")
print(f"  有配對的秒數: {len(section_pairs)}")

# RGB 時間對齊
rgb_errors = [p['rgb_error_ms'] for p in pairs_adaptive]
print(f"\nRGB 時間對齊:")
print(f"  誤差 < 20ms: {sum(1 for e in rgb_errors if e < 20)} ({sum(1 for e in rgb_errors if e < 20)/len(rgb_errors)*100:.1f}%)")
print(f"  誤差 < 40ms: {sum(1 for e in rgb_errors if e < 40)} ({sum(1 for e in rgb_errors if e < 40)/len(rgb_errors)*100:.1f}%)")
print(f"  誤差 > 100ms: {sum(1 for e in rgb_errors if e > 100)} ({sum(1 for e in rgb_errors if e > 100)/len(rgb_errors)*100:.1f}%)")

print("="*70)


配對品質驗證

每秒配對數分佈:
  平均: 7.8
  最小: 7
  最大: 8
  有配對的秒數: 659

RGB 時間對齊:
  誤差 < 20ms: 5084 (99.1%)
  誤差 < 40ms: 5132 (100.0%)
  誤差 > 100ms: 0 (0.0%)


---
## Step 7: 視覺化分析

In [ ]:
# 可視化結果
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. RGB 誤差分布
axes[0, 0].hist(rgb_errors, bins=50, edgecolor='black', alpha=0.7, color='#51cf66')
axes[0, 0].axvline(np.mean(rgb_errors), color='red', linestyle='--', linewidth=2, 
                   label=f'Mean: {np.mean(rgb_errors):.2f}ms')
axes[0, 0].set_xlabel('RGB Error (ms)')
axes[0, 0].set_ylabel('Count')
axes[0, 0].set_title('RGB Time Alignment Error Distribution')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. 每秒配對數（前 100 秒）
secs = sorted(section_pairs.keys())
counts = [section_pairs[sec] for sec in secs]
plot_secs = secs[:100]
plot_counts = counts[:100]

axes[0, 1].plot(plot_secs, plot_counts, marker='o', markersize=4, linewidth=1)
axes[0, 1].axhline(NORMAL_FRAMES_PER_SEC, color='red', linestyle='--', 
                   label=f'Target: {NORMAL_FRAMES_PER_SEC} pairs/sec')
axes[0, 1].set_xlabel('Second')
axes[0, 1].set_ylabel('Pairs Count')
axes[0, 1].set_title('Pairs Distribution per Second (First 100s)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# 3. 配對計數直方圖
axes[1, 0].hist(pair_counts, bins=range(0, max(pair_counts)+2), edgecolor='black', 
                alpha=0.7, color='#74c0fc')
axes[1, 0].set_xlabel('Pairs per Second')
axes[1, 0].set_ylabel('Count')
axes[1, 0].set_title('Distribution of Pairs per Second')
axes[1, 0].grid(True, alpha=0.3)

# 4. 重新分配統計
if redistribution_log:
    redistribution_from = {}
    for log in redistribution_log:
        if log['from_sec'] not in redistribution_from:
            redistribution_from[log['from_sec']] = 0
        redistribution_from[log['from_sec']] += 1
    
    from_secs = sorted(redistribution_from.keys())[:20]
    from_counts = [redistribution_from[sec] for sec in from_secs]
    
    axes[1, 1].bar(range(len(from_secs)), from_counts, color='#ffa94d', edgecolor='black')
    axes[1, 1].set_xticks(range(len(from_secs)))
    axes[1, 1].set_xticklabels([f"Sec {sec}" for sec in from_secs], rotation=45)
    axes[1, 1].set_ylabel('Redistributed Frames')
    axes[1, 1].set_title('Top 20 Burst Seconds (Redistributed Frames)')
    axes[1, 1].grid(True, alpha=0.3, axis='y')
else:
    axes[1, 1].text(0.5, 0.5, 'No Redistribution Needed', 
                   ha='center', va='center', transform=axes[1, 1].transAxes)
    axes[1, 1].set_title('Redistribution Log')

plt.tight_layout()
save_path = ADAPTIVE_DIR / 'adaptive_pairing_analysis.png'
plt.savefig(save_path, dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✅ 視覺化已儲存: {save_path}")

---
## Step 8: 翻轉檢測

In [ ]:
def check_flip_orientation(pairs: List[Dict], avi_path: Path, num_samples: int = 6) -> None:
    """
    檢查是否需要左右翻轉
    """
    import random
    
    if not pairs:
        print("❌ 無配對資料可檢查")
        return
    
    print(f"\n{'='*70}")
    print(f"翻轉方向檢測")
    print(f"{'='*70}")
    
    # 隨機取樣
    sample_size = min(num_samples, len(pairs))
    sample_indices = random.sample(range(len(pairs)), sample_size)
    sample_pairs = [pairs[i] for i in sample_indices]
    
    cap = cv2.VideoCapture(str(avi_path))
    
    # 4 種組合
    fig, axes = plt.subplots(num_samples, 8, figsize=(24, num_samples*3))
    
    for row_idx, sample_pair in enumerate(sample_pairs):
        thermal = sample_pair['thermal']['image']
        
        # 讀取 RGB
        cap.set(cv2.CAP_PROP_POS_FRAMES, sample_pair['rgb_frame_idx'])
        ret, rgb = cap.read()
        
        if not ret:
            continue
        
        rgb = cv2.cvtColor(rgb, cv2.COLOR_BGR2RGB)
        
        # 4 種組合
        combinations = [
            (thermal, rgb, "原始", "Both Original"),
            (cv2.flip(thermal, 1), rgb, "T翻轉", "Thermal Flipped"),
            (thermal, cv2.flip(rgb, 1), "R翻轉", "RGB Flipped"),
            (cv2.flip(thermal, 1), cv2.flip(rgb, 1), "都翻轉", "Both Flipped")
        ]
        
        for col_idx, (t, r, label, desc) in enumerate(combinations):
            t_up = cv2.resize(t, (r.shape[1], r.shape[0]), interpolation=cv2.INTER_CUBIC)
            
            # Thermal
            axes[row_idx, col_idx*2].imshow(t_up, cmap='hot')
            axes[row_idx, col_idx*2].set_title(f"{label}\nThermal", fontsize=10)
            axes[row_idx, col_idx*2].axis('off')
            
            # RGB
            axes[row_idx, col_idx*2+1].imshow(r)
            axes[row_idx, col_idx*2+1].set_title(f"{label}\nRGB", fontsize=10)
            axes[row_idx, col_idx*2+1].axis('off')
    
    cap.release()
    
    plt.suptitle('翻轉方向檢測 - 請選擇畫面對應最好的組合', fontsize=14, fontweight='bold')
    plt.tight_layout()
    
    save_path = DIAGNOSTIC_DIR / 'flip_comparison.png'
    plt.savefig(save_path, dpi=120, bbox_inches='tight')
    print(f"\n✅ 翻轉比對圖已儲存: {save_path}")
    plt.show()
    
    print(f"\n📝 請檢查圖片，判斷哪種組合正確:")
    print(f"  選項 1 - 原始: FLIP_THERMAL=False, FLIP_RGB=False")
    print(f"  選項 2 - T翻轉: FLIP_THERMAL=True, FLIP_RGB=False")
    print(f"  選項 3 - R翻轉: FLIP_THERMAL=False, FLIP_RGB=True")
    print(f"  選項 4 - 都翻轉: FLIP_THERMAL=True, FLIP_RGB=True")
    print(f"\n確認後請修改 Step 1 的 FLIP_THERMAL 和 FLIP_RGB 參數")
    print("="*70)

# 執行翻轉檢測
check_flip_orientation(pairs_adaptive, AVI_FILE)

---
## Step 9: 儲存配對結果

In [44]:
# 保存配對結果為 JSON 和 CSV
pairs_export = []
for pair in pairs_adaptive:
    pairs_export.append({
        'pair_id': pair['pair_id'],
        'rgb_frame_idx': pair['rgb_frame_idx'],
        'timestamp': pair['timestamp'].isoformat(),
        'section': pair['section'],
        'rgb_error_ms': pair['rgb_error_ms'],
        'label': 'thermal',
        'modality': 'thermal-rgb'
    })

# 儲存 JSON
json_path = ADAPTIVE_DIR / 'pairs_mapping.json'
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(pairs_export, f, indent=2, ensure_ascii=False)

# 儲存 CSV
csv_path = ADAPTIVE_DIR / 'pairs_mapping.csv'
df_pairs = pd.DataFrame(pairs_export)
df_pairs.to_csv(csv_path, index=False, encoding='utf-8-sig')

print(f"\n{'='*70}")
print(f"配對結果已儲存")
print(f"{'='*70}")
print(f"  JSON: {json_path}")
print(f"  CSV: {csv_path}")
print(f"  總配對數: {len(pairs_export):,}")
print("="*70)


配對結果已儲存
  JSON: /home/gary/claude4.5/output/adaptive_pairing/pairs_mapping.json
  CSV: /home/gary/claude4.5/output/adaptive_pairing/pairs_mapping.csv
  總配對數: 5,132


---
## Step 10: 匯出 RGB 圖片供 LabelMe 標註（多核加速）

In [45]:
def export_rgb_batch(args: Tuple) -> int:
    """
    批次匯出 RGB 圖片（多核處理用）
    """
    pairs_batch, avi_path, output_dir, flip_rgb = args
    
    cap = cv2.VideoCapture(str(avi_path))
    exported_count = 0
    
    for pair in pairs_batch:
        cap.set(cv2.CAP_PROP_POS_FRAMES, pair['rgb_frame_idx'])
        ret, frame = cap.read()
        
        if not ret:
            continue
        
        if flip_rgb:
            frame = cv2.flip(frame, 1)
        
        # 儲存為 JPG
        filename = f"frame_{pair['pair_id'].replace('pair_', '')}.jpg"
        save_path = output_dir / filename
        cv2.imwrite(str(save_path), frame, [cv2.IMWRITE_JPEG_QUALITY, 95])
        
        exported_count += 1
    
    cap.release()
    return exported_count

# 建立 LabelMe 目錄
images_dir = LABELME_DIR / 'images'
annotations_dir = LABELME_DIR / 'annotations'
images_dir.mkdir(exist_ok=True)
annotations_dir.mkdir(exist_ok=True)

# 分批處理
batches = [pairs_adaptive[i:i+CHUNK_SIZE] for i in range(0, len(pairs_adaptive), CHUNK_SIZE)]
args_list = [(batch, AVI_FILE, images_dir, FLIP_RGB) for batch in batches]

print(f"\n{'='*70}")
print(f"匯出 RGB 圖片供 LabelMe 標註")
print(f"{'='*70}")
print(f"  總配對數: {len(pairs_adaptive):,}")
print(f"  翻轉 RGB: {FLIP_RGB}")
print(f"  多核處理: {NUM_WORKERS} workers")

# 多核執行
print(f"\n開始匯出...")
with ProcessPoolExecutor(max_workers=NUM_WORKERS) as executor:
    results = list(tqdm(
        executor.map(export_rgb_batch, args_list),
        total=len(batches),
        desc="匯出 RGB"
    ))

total_exported = sum(results)

print(f"\n✅ 匯出完成")
print(f"  成功匯出: {total_exported:,} 張圖片")
print(f"  位置: {images_dir}")
print("="*70)


匯出 RGB 圖片供 LabelMe 標註
  總配對數: 5,132
  翻轉 RGB: False
  多核處理: 3 workers

開始匯出...


匯出 RGB: 100%|██████████| 52/52 [00:22<00:00,  2.34it/s]


✅ 匯出完成
  成功匯出: 5,132 張圖片
  位置: /home/gary/claude4.5/output/labelme_project/images


In [46]:
def save_aligned_thermal_npy(pairs: List[Dict], output_dir: Path, flip: bool = False):
    """
    儲存對齊後的 Thermal 為 NPY
    """
    output_dir.mkdir(exist_ok=True)
    
    print(f"\n{'='*70}")
    print(f"儲存 Thermal 為 NPY")
    print(f"{'='*70}")
    
    for pair in tqdm(pairs, desc="儲存 Thermal"):
        thermal_img = pair['thermal']['image']  # uint8 (24, 32)
        
        if flip:
            thermal_img = cv2.flip(thermal_img, 1)
        
        # 方案 A: 儲存原始 uint8
        save_path = output_dir / f"{pair['pair_id']}_thermal.npy"
        np.save(save_path, thermal_img)
        
        # 方案 B: 儲存正規化 float32（可選）
        # thermal_normalized = thermal_img.astype(np.float32) / 255.0
        # np.save(save_path, thermal_normalized)
    
    print(f"\n✅ 完成，儲存於 {output_dir}")

# 執行
THERMAL_OUT = ALIGNED_DIR / 'thermal'
save_aligned_thermal_npy(pairs_adaptive, THERMAL_OUT, FLIP_THERMAL)


儲存 Thermal 為 NPY


儲存 Thermal: 100%|██████████| 5132/5132 [00:00<00:00, 12858.58it/s]


✅ 完成，儲存於 /home/gary/claude4.5/output/aligned_dataset/thermal


---
## Step 11: 建立 LabelMe 專案環境

In [47]:
# 類別定義
pose_classes = ['lying', 'sitting', 'fallen', 'empty', 'uncertain']

classes_file = LABELME_DIR / 'classes.txt'
with open(classes_file, 'w', encoding='utf-8') as f:
    for cls in pose_classes:
        f.write(f"{cls}\n")

print(f"✅ 類別定義已建立: {classes_file}")
print(f"\n標註類別:")
for cls in pose_classes:
    print(f"  - {cls}")

# README
readme_content = f"""# LabelMe 標註專案 - 病床姿勢辨識

## 專案資訊
- 總圖片數: {len(pairs_adaptive):,}
- 建立時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
- 使用者: rochi190

## 開始標註
```bash
cd {LABELME_DIR}
labelme images --labels classes.txt --output annotations
```

## 標註類別
- **lying**: 躺著（平躺在床上）
- **sitting**: 坐著（坐在床上或床邊）
- **fallen**: 跌倒（跌落床下）
- **empty**: 空床（無人）
- **uncertain**: 不確定（畫面不清楚）

## 標註步驟
1. LabelMe 視窗會自動開啟
2. 點擊「Create Polygons」或按 `Ctrl+N`
3. 用滑鼠框選**人體區域**（盡量完整）
4. 從下拉選單選擇姿勢類別
5. 按 `Ctrl+S` 儲存
6. 按 `D` 下一張圖片

## 快捷鍵
- `Ctrl+N`: 建立新標註
- `Ctrl+S`: 儲存
- `D`: 下一張
- `A`: 上一張
- `Ctrl+Z`: 復原

## 標註進度追蹤
```bash
ls annotations/*.json | wc -l
```

## 資料來源
- 配對策略: Adaptive Pairing（處理不固定 FPS）
- 配對數據: {ADAPTIVE_DIR / 'pairs_mapping.json'}
"""

readme_path = LABELME_DIR / 'README.md'
with open(readme_path, 'w', encoding='utf-8') as f:
    f.write(readme_content)

print(f"\n✅ 使用說明已建立: {readme_path}")

✅ 類別定義已建立: /home/gary/claude4.5/output/labelme_project/classes.txt

標註類別:
  - lying
  - sitting
  - fallen
  - empty
  - uncertain

✅ 使用說明已建立: /home/gary/claude4.5/output/labelme_project/README.md


---
## Step 12: 最終總結

In [48]:
# 最終總結
print(f"\n{'='*70}")
print(f"資料準備完成 - 最終總結")
print(f"{'='*70}")

print(f"\n📊 輸入資料:")
print(f"  Thermal 幀數: {len(therm_data):,}")
print(f"  RGB 幀數: {frame_count:,} @ {fps:.2f}fps")
print(f"  時長: {duration:.2f} 秒")

print(f"\n🔍 分佈分析:")
section_counts = {sec: len(frames) for sec, frames in sections.items()}
counts_list = list(section_counts.values())
print(f"  總秒數: {len(sections)}")
print(f"  最小幀數/秒: {min(counts_list)}")
print(f"  最大幀數/秒: {max(counts_list)}")
print(f"  平均幀數/秒: {np.mean(counts_list):.1f}")

burst_count = len([c for c in counts_list if c > NORMAL_FRAMES_PER_SEC])
dropout_count = len([c for c in counts_list if c < NORMAL_FRAMES_PER_SEC])
print(f"\n⚠️ 異常檢測:")
print(f"  Burst seconds (>{NORMAL_FRAMES_PER_SEC} 張/秒): {burst_count}")
print(f"  Dropout seconds (<{NORMAL_FRAMES_PER_SEC} 張/秒): {dropout_count}")
print(f"  重新分配幀數: {len(redistribution_log)}")

print(f"\n✅ 配對結果:")
print(f"  總配對數: {len(pairs_adaptive):,}")
print(f"  成功率: {len(pairs_adaptive) / len(therm_data) * 100:.1f}%")
print(f"  平均配對數/秒: {np.mean(pair_counts):.1f}")

print(f"\n⏱️ RGB 時間精度:")
print(f"  平均誤差: {np.mean(rgb_errors):.2f}ms")
print(f"  最大誤差: {np.max(rgb_errors):.2f}ms")
print(f"  誤差 < 20ms: {sum(1 for e in rgb_errors if e < 20)} ({sum(1 for e in rgb_errors if e < 20)/len(rgb_errors)*100:.1f}%)")

print(f"\n📁 輸出檔案:")
print(f"  配對分析圖: {ADAPTIVE_DIR / 'adaptive_pairing_analysis.png'}")
print(f"  翻轉比對圖: {DIAGNOSTIC_DIR / 'flip_comparison.png'}")
print(f"  配對 JSON: {ADAPTIVE_DIR / 'pairs_mapping.json'}")
print(f"  配對 CSV: {ADAPTIVE_DIR / 'pairs_mapping.csv'}")
print(f"  LabelMe 圖片: {LABELME_DIR / 'images'} ({total_exported:,} 張)")
print(f"  LabelMe README: {LABELME_DIR / 'README.md'}")

print(f"\n🎯 下一步:")
print(f"\n1. 檢查翻轉方向:")
print(f"   - 查看 {DIAGNOSTIC_DIR / 'flip_comparison.png'}")
print(f"   - 如需調整，修改 Step 1 的 FLIP_THERMAL 和 FLIP_RGB")
print(f"   - 重新執行 Step 10-11")

print(f"\n2. 開始 LabelMe 標註:")
print(f"   cd {LABELME_DIR}")
print(f"   labelme images --labels classes.txt --output annotations")

print(f"\n3. 標註完成後:")
print(f"   - 執行 02_convert_labelme_to_dataset.ipynb")
print(f"   - 執行 03_train_semantic_guided_sr.ipynb")

print(f"\n✨ 資料準備完成！Ready for labeling!")
print("="*70)


資料準備完成 - 最終總結

📊 輸入資料:
  Thermal 幀數: 5,318
  RGB 幀數: 16,284 @ 24.67fps
  時長: 660.07 秒

🔍 分佈分析:
  總秒數: 677
  最小幀數/秒: 6
  最大幀數/秒: 8
  平均幀數/秒: 7.9

⚠️ 異常檢測:
  Burst seconds (>8 張/秒): 0
  Dropout seconds (<8 張/秒): 97
  重新分配幀數: 85

✅ 配對結果:
  總配對數: 5,132
  成功率: 96.5%
  平均配對數/秒: 7.8

⏱️ RGB 時間精度:
  平均誤差: 10.10ms
  最大誤差: 20.20ms
  誤差 < 20ms: 5084 (99.1%)

📁 輸出檔案:
  配對分析圖: /home/gary/claude4.5/output/adaptive_pairing/adaptive_pairing_analysis.png
  翻轉比對圖: /home/gary/claude4.5/output/diagnostics/flip_comparison.png
  配對 JSON: /home/gary/claude4.5/output/adaptive_pairing/pairs_mapping.json
  配對 CSV: /home/gary/claude4.5/output/adaptive_pairing/pairs_mapping.csv
  LabelMe 圖片: /home/gary/claude4.5/output/labelme_project/images (5,132 張)
  LabelMe README: /home/gary/claude4.5/output/labelme_project/README.md

🎯 下一步:

1. 檢查翻轉方向:
   - 查看 /home/gary/claude4.5/output/diagnostics/flip_comparison.png
   - 如需調整，修改 Step 1 的 FLIP_THERMAL 和 FLIP_RGB
   - 重新執行 Step 10-11

2. 開始 LabelMe 標註:
   cd /home/gary/cl

In [49]:
# ========================================
# 檢查工具：驗證 RGB 配對連續性
# ========================================

def check_rgb_frame_continuity(pairs: List[Dict]) -> Dict:
    """
    檢查配對後的 RGB 幀索引是否連續
    
    Returns:
        統計資訊字典
    """
    print(f"\n{'='*70}")
    print(f"RGB 配對連續性檢查")
    print(f"{'='*70}")
    
    # 取得所有 RGB 幀索引
    rgb_indices = sorted([p['rgb_frame_idx'] for p in pairs])
    
    print(f"\n📊 基本統計:")
    print(f"  總配對數: {len(rgb_indices):,}")
    print(f"  RGB 幀範圍: {rgb_indices[0]} ~ {rgb_indices[-1]}")
    print(f"  幀跨度: {rgb_indices[-1] - rgb_indices[0] + 1}")
    
    # 檢查間隔
    gaps = []
    duplicates = []
    
    for i in range(len(rgb_indices) - 1):
        gap = rgb_indices[i+1] - rgb_indices[i]
        
        if gap == 0:
            # 重複
            duplicates.append({
                'index': rgb_indices[i],
                'position': i
            })
        elif gap > 1:
            # 有跳躍
            gaps.append({
                'from': rgb_indices[i],
                'to': rgb_indices[i+1],
                'gap_size': gap - 1,
                'position': i
            })
    
    # 間隔分佈
    intervals = [rgb_indices[i+1] - rgb_indices[i] for i in range(len(rgb_indices) - 1)]
    interval_counts = {}
    for interval in intervals:
        if interval not in interval_counts:
            interval_counts[interval] = 0
        interval_counts[interval] += 1
    
    print(f"\n📏 間隔分佈:")
    for interval in sorted(interval_counts.keys())[:10]:
        count = interval_counts[interval]
        percentage = count / len(intervals) * 100
        print(f"  間隔 {interval:3d} 幀: {count:5d} 次 ({percentage:5.1f}%)")
    
    if len(interval_counts) > 10:
        print(f"  ... 還有 {len(interval_counts) - 10} 種間隔")
    
    # Gap 分析
    print(f"\n🕳️ 間隙 (Gap) 分析:")
    if gaps:
        print(f"  檢測到 {len(gaps)} 個間隙")
        print(f"\n  前 10 個間隙:")
        for gap in gaps[:10]:
            print(f"    幀 {gap['from']:5d} → {gap['to']:5d} (跳過 {gap['gap_size']} 幀)")
        
        if len(gaps) > 10:
            print(f"    ... 還有 {len(gaps) - 10} 個間隙")
        
        # 間隙統計
        gap_sizes = [g['gap_size'] for g in gaps]
        print(f"\n  間隙統計:")
        print(f"    最小間隙: {min(gap_sizes)} 幀")
        print(f"    最大間隙: {max(gap_sizes)} 幀")
        print(f"    平均間隙: {np.mean(gap_sizes):.1f} 幀")
    else:
        print(f"  ✅ 無間隙（完全連續）")
    
    # 重複檢查
    print(f"\n🔄 重複檢查:")
    if duplicates:
        print(f"  ⚠️ 檢測到 {len(duplicates)} 個重複幀")
        print(f"\n  前 10 個重複:")
        for dup in duplicates[:10]:
            print(f"    幀 {dup['index']} 被重複使用")
        
        if len(duplicates) > 10:
            print(f"    ... 還有 {len(duplicates) - 10} 個重複")
    else:
        print(f"  ✅ 無重複")
    
    # 連續性評分
    total_frames_covered = rgb_indices[-1] - rgb_indices[0] + 1
    continuity_score = len(rgb_indices) / total_frames_covered * 100
    
    print(f"\n📈 連續性評分:")
    print(f"  覆蓋幀數: {len(rgb_indices):,} / {total_frames_covered:,}")
    print(f"  連續性: {continuity_score:.1f}%")
    
    if continuity_score > 90:
        print(f"  評級: ✅ 優秀")
    elif continuity_score > 70:
        print(f"  評級: 🟡 良好")
    else:
        print(f"  評級: ⚠️ 需要改善")
    
    print("="*70)
    
    return {
        'total_pairs': len(rgb_indices),
        'frame_range': (rgb_indices[0], rgb_indices[-1]),
        'gaps': gaps,
        'duplicates': duplicates,
        'continuity_score': continuity_score,
        'interval_distribution': interval_counts
    }

# 執行檢查
continuity_stats = check_rgb_frame_continuity(pairs_adaptive)


RGB 配對連續性檢查

📊 基本統計:
  總配對數: 5,132
  RGB 幀範圍: 31 ~ 16264
  幀跨度: 16234

📏 間隔分佈:
  間隔   0 幀:  4473 次 ( 87.2%)
  間隔  24 幀:   217 次 (  4.2%)
  間隔  25 幀:   441 次 (  8.6%)

🕳️ 間隙 (Gap) 分析:
  檢測到 658 個間隙

  前 10 個間隙:
    幀    31 →    56 (跳過 24 幀)
    幀    56 →    81 (跳過 24 幀)
    幀    81 →   105 (跳過 23 幀)
    幀   105 →   130 (跳過 24 幀)
    幀   130 →   155 (跳過 24 幀)
    幀   155 →   179 (跳過 23 幀)
    幀   179 →   204 (跳過 24 幀)
    幀   204 →   229 (跳過 24 幀)
    幀   229 →   253 (跳過 23 幀)
    幀   253 →   278 (跳過 24 幀)
    ... 還有 648 個間隙

  間隙統計:
    最小間隙: 23 幀
    最大間隙: 24 幀
    平均間隙: 23.7 幀

🔄 重複檢查:
  ⚠️ 檢測到 4473 個重複幀

  前 10 個重複:
    幀 31 被重複使用
    幀 31 被重複使用
    幀 31 被重複使用
    幀 31 被重複使用
    幀 56 被重複使用
    幀 56 被重複使用
    幀 56 被重複使用
    幀 56 被重複使用
    幀 56 被重複使用
    幀 56 被重複使用
    ... 還有 4463 個重複

📈 連續性評分:
  覆蓋幀數: 5,132 / 16,234
  連續性: 31.6%
  評級: ⚠️ 需要改善


In [ ]:
# ========================================
# 視覺化：RGB 幀索引分佈
# ========================================

def visualize_rgb_frame_distribution(pairs: List[Dict], save_dir: Path):
    """
    視覺化 RGB 幀索引的分佈和連續性
    """
    rgb_indices = sorted([p['rgb_frame_idx'] for p in pairs])
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    # 1. RGB 幀索引序列圖
    axes[0, 0].plot(range(len(rgb_indices)), rgb_indices, 'b-', linewidth=0.5, alpha=0.7)
    axes[0, 0].scatter(range(len(rgb_indices)), rgb_indices, c='blue', s=1, alpha=0.3)
    axes[0, 0].set_xlabel('配對索引')
    axes[0, 0].set_ylabel('RGB 幀索引')
    axes[0, 0].set_title('RGB 幀索引序列（應該是一條斜直線）')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 幀間隔分佈
    intervals = [rgb_indices[i+1] - rgb_indices[i] for i in range(len(rgb_indices) - 1)]
    axes[0, 1].hist(intervals, bins=50, edgecolor='black', alpha=0.7, color='green')
    axes[0, 1].axvline(1, color='red', linestyle='--', linewidth=2, label='理想值: 1')
    axes[0, 1].set_xlabel('幀間隔')
    axes[0, 1].set_ylabel('出現次數')
    axes[0, 1].set_title('RGB 幀間隔分佈（大部分應該是 1）')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. 局部放大（前 100 對）
    sample_size = min(100, len(rgb_indices))
    axes[1, 0].plot(range(sample_size), rgb_indices[:sample_size], 'bo-', markersize=4)
    axes[1, 0].set_xlabel('配對索引')
    axes[1, 0].set_ylabel('RGB 幀索引')
    axes[1, 0].set_title(f'局部放大（前 {sample_size} 對）')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 標註間隙
    for i in range(min(sample_size - 1, len(rgb_indices) - 1)):
        gap = rgb_indices[i+1] - rgb_indices[i]
        if gap > 1:
            axes[1, 0].axvspan(i, i+1, alpha=0.3, color='red')
            axes[1, 0].text(i+0.5, rgb_indices[i], f'+{gap-1}', 
                          ha='center', va='bottom', fontsize=8, color='red')
    
    # 4. 累積覆蓋率
    expected_frames = list(range(rgb_indices[0], rgb_indices[-1] + 1))
    coverage = []
    for frame in expected_frames:
        if frame in rgb_indices:
            coverage.append(1)
        else:
            coverage.append(0)
    
    cumulative_coverage = np.cumsum(coverage) / np.arange(1, len(coverage) + 1) * 100
    
    axes[1, 1].plot(expected_frames, cumulative_coverage, 'g-', linewidth=2)
    axes[1, 1].axhline(100, color='red', linestyle='--', label='理想值: 100%')
    axes[1, 1].set_xlabel('RGB 幀索引')
    axes[1, 1].set_ylabel('累積覆蓋率 (%)')
    axes[1, 1].set_title('RGB 幀累積覆蓋率')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    save_path = save_dir / 'rgb_frame_continuity.png'
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    print(f"\n✅ 視覺化已儲存: {save_path}")
    plt.show()

# 執行視覺化
visualize_rgb_frame_distribution(pairs_adaptive, DIAGNOSTIC_DIR)

In [51]:
# ========================================
# 檢查 LabelMe 圖片順序
# ========================================

def check_labelme_image_order(labelme_dir: Path, pairs: List[Dict]):
    """
    檢查 LabelMe 專案中圖片的順序
    """
    images_dir = labelme_dir / 'images'
    
    if not images_dir.exists():
        print(f"❌ 圖片目錄不存在: {images_dir}")
        return
    
    print(f"\n{'='*70}")
    print(f"LabelMe 圖片順序檢查")
    print(f"{'='*70}")
    
    # 取得所有圖片
    image_files = sorted(images_dir.glob('*.jpg'))
    
    print(f"\n📁 圖片目錄: {images_dir}")
    print(f"  總圖片數: {len(image_files)}")
    
    # 解析圖片檔名中的 pair_id
    print(f"\n前 20 個圖片:")
    for i, img_path in enumerate(image_files[:20], 1):
        # 假設檔名格式: frame_00000.jpg
        filename = img_path.name
        
        # 找對應的 pair
        pair_id_str = filename.replace('frame_', '').replace('.jpg', '')
        pair_id = f"pair_{pair_id_str}"
        
        # 找 RGB 幀索引
        matching_pair = next((p for p in pairs if p['pair_id'] == pair_id), None)
        
        if matching_pair:
            rgb_idx = matching_pair['rgb_frame_idx']
            print(f"  {i:3d}. {filename:20s} → pair_id: {pair_id} → RGB 幀: {rgb_idx}")
        else:
            print(f"  {i:3d}. {filename:20s} → ⚠️ 找不到配對")
    
    if len(image_files) > 20:
        print(f"  ... 還有 {len(image_files) - 20} 個")
    
    # 檢查檔名順序 vs RGB 幀順序
    print(f"\n🔍 順序一致性檢查:")
    
    inconsistencies = []
    
    for i in range(min(len(image_files) - 1, 100)):
        img1 = image_files[i]
        img2 = image_files[i+1]
        
        # 解析 pair_id
        pair_id1 = f"pair_{img1.name.replace('frame_', '').replace('.jpg', '')}"
        pair_id2 = f"pair_{img2.name.replace('frame_', '').replace('.jpg', '')}"
        
        # 找 RGB 幀索引
        pair1 = next((p for p in pairs if p['pair_id'] == pair_id1), None)
        pair2 = next((p for p in pairs if p['pair_id'] == pair_id2), None)
        
        if pair1 and pair2:
            rgb1 = pair1['rgb_frame_idx']
            rgb2 = pair2['rgb_frame_idx']
            
            # 檢查是否遞增
            if rgb2 < rgb1:
                inconsistencies.append({
                    'position': i,
                    'file1': img1.name,
                    'file2': img2.name,
                    'rgb1': rgb1,
                    'rgb2': rgb2,
                    'problem': '順序錯誤（RGB 幀倒退）'
                })
            elif rgb2 - rgb1 > 10:
                inconsistencies.append({
                    'position': i,
                    'file1': img1.name,
                    'file2': img2.name,
                    'rgb1': rgb1,
                    'rgb2': rgb2,
                    'problem': f'跳躍過大（跳過 {rgb2 - rgb1 - 1} 幀）'
                })
    
    if inconsistencies:
        print(f"  ⚠️ 發現 {len(inconsistencies)} 個順序問題:")
        for issue in inconsistencies[:10]:
            print(f"    位置 {issue['position']}: {issue['file1']} (幀{issue['rgb1']}) → {issue['file2']} (幀{issue['rgb2']})")
            print(f"      問題: {issue['problem']}")
        
        if len(inconsistencies) > 10:
            print(f"    ... 還有 {len(inconsistencies) - 10} 個問題")
    else:
        print(f"  ✅ 前 100 張圖片順序正確")
    
    print("="*70)

# 執行檢查
check_labelme_image_order(LABELME_DIR, pairs_adaptive)


LabelMe 圖片順序檢查

📁 圖片目錄: /home/gary/claude4.5/output/labelme_project/images
  總圖片數: 5132

前 20 個圖片:
    1. frame_00000.jpg      → pair_id: pair_00000 → RGB 幀: 31
    2. frame_00001.jpg      → pair_id: pair_00001 → RGB 幀: 31
    3. frame_00002.jpg      → pair_id: pair_00002 → RGB 幀: 31
    4. frame_00003.jpg      → pair_id: pair_00003 → RGB 幀: 31
    5. frame_00004.jpg      → pair_id: pair_00004 → RGB 幀: 31
    6. frame_00005.jpg      → pair_id: pair_00005 → RGB 幀: 56
    7. frame_00006.jpg      → pair_id: pair_00006 → RGB 幀: 56
    8. frame_00007.jpg      → pair_id: pair_00007 → RGB 幀: 56
    9. frame_00008.jpg      → pair_id: pair_00008 → RGB 幀: 56
   10. frame_00009.jpg      → pair_id: pair_00009 → RGB 幀: 56
   11. frame_00010.jpg      → pair_id: pair_00010 → RGB 幀: 56
   12. frame_00011.jpg      → pair_id: pair_00011 → RGB 幀: 56
   13. frame_00012.jpg      → pair_id: pair_00012 → RGB 幀: 81
   14. frame_00013.jpg      → pair_id: pair_00013 → RGB 幀: 81
   15. frame_00014.jpg      → pa